In [1]:
!ls

README.md	    data		model.pth    train.ipynb
__pycache__	    data_split.ipynb	model_save   train.py
balanced_train.csv  dataset		old	     utils
balanced_val.csv    json_parsing.ipynb	sample_data  validation.csv
checkData.ipynb     labeling.ipynb	test.ipynb   wandb
config.py	    model		train.csv


In [2]:
!nvidia-smi

Mon Apr 15 06:02:55 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.239.06   Driver Version: 470.239.06   CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:0E:00.0 Off |                  Off |
| 32%   53C    P5    81W / 300W |  15701MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import wandb

# init
wandb.init(project='Facial Expression Recognition', 
           group='yuhohyun',
           name='2024_04_15(1)',
           notes='shuffled, Epoch=20, Batch size=16, lr=0.0001',
           tags='',
           entity='yuhohyun')

# config
wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 20,
  "batch_size": 32
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yuhohyun. Use `wandb login --relogin` to force relogin


In [4]:
import torch
print(torch.cuda.is_available())  # True가 출력되면 CUDA 사용 가능
print(torch.version.cuda)  # 설치된 PyTorch가 사용하는 CUDA 버전 출력

True
11.7


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision import datasets, transforms
from torchmetrics import Precision, Recall, F1Score
import datetime
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score

from model.model import PAtt_Lite
import config
from utils.data_utils import CustomImageDataset

def load_data(data_dir, batch_size):
    train_transform = transforms.Compose([
        transforms.Resize(config.IMG_SHAPE[:2]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize(config.IMG_SHAPE[:2]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = CustomImageDataset(
        annotations_file=os.path.join(data_dir, 'balanced_train.csv'), 
        img_dir=os.path.join(data_dir, ''), 
        transform=train_transform)
    val_dataset = CustomImageDataset(
        annotations_file=os.path.join(data_dir, 'balanced_val.csv'), 
        img_dir=os.path.join(data_dir, ''), 
        transform=val_transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    return train_loader, val_loader

In [6]:
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Metrics 초기화
    precision = Precision(num_classes=config.NUM_CLASSES, average='macro', task='multiclass').to(device)
    recall = Recall(num_classes=config.NUM_CLASSES, average='macro', task='multiclass').to(device)
    f1 = F1Score(num_classes=config.NUM_CLASSES, average='macro', task='multiclass').to(device)

    # WandB에 모델 추적 시작하기
    wandb.watch(model, criterion, log="all", log_freq=16)
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch'):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            precision.update(outputs, labels)
            recall.update(outputs, labels)
            f1.update(outputs, labels)
        
        train_loss = running_loss / len(train_loader.dataset)
        train_precision = precision.compute()
        train_recall = recall.compute()
        train_f1 = f1.compute()
        
        # 메트릭 초기화
        precision.reset()
        recall.reset()
        f1.reset()
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                precision.update(outputs, labels)
                recall.update(outputs, labels)
                f1.update(outputs, labels)
        
        val_loss = val_loss / len(val_loader.dataset)
        val_precision = precision.compute()
        val_recall = recall.compute()
        val_f1 = f1.compute()
        
        # 스케줄러 스텝
        scheduler.step()
        
        # WandB에 기록
        wandb.log({"epoch": epoch + 1, 
           "train_loss": train_loss, "val_loss": val_loss,
           "train_precision": train_precision, "val_precision": val_precision,
           "train_recall": train_recall, "val_recall": val_recall,
           "train_f1": train_f1, "val_f1": val_f1})
        
        print(f'\nEpoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        print(f'Train Precision: {train_precision:.2f}, Train Recall: {train_recall:.2f}, Train F1: {train_f1:.2f}')
        print(f'Validation Precision: {val_precision:.2f}, Validation Recall: {val_recall:.2f}, Validation F1: {val_f1:.2f}')


In [7]:
def main():
    train_loader, val_loader = load_data('', config.BATCH_SIZE)
    model = PAtt_Lite(config.NUM_CLASSES)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.TRAIN_LR)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    
    train_and_validate(model, train_loader, val_loader, criterion, optimizer, scheduler, config.TRAIN_EPOCH)

    torch.save(model.state_dict(), 'model.pth')

if __name__ == "__main__":
    main()


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/20: 100% 3631/3631 [17:21<00:00,  3.49batch/s]



Epoch 1/20, Train Loss: 0.7710, Validation Loss: 0.8065
Train Precision: 0.58, Train Recall: 0.57, Train F1: 0.56
Validation Precision: 0.63, Validation Recall: 0.63, Validation F1: 0.59


Epoch 2/20:  83% 2998/3631 [14:12<02:44,  3.85batch/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Epoch 10/20: 100% 3631/3631 [19:20<00:00,  3.13batch/s]



Epoch 10/20, Train Loss: 0.2426, Validation Loss: 0.6314
Train Precision: 0.86, Train Recall: 0.86, Train F1: 0.86
Validation Precision: 0.78, Validation Recall: 0.77, Validation F1: 0.77


Epoch 11/20: 100% 3631/3631 [19:23<00:00,  3.12batch/s]


...


Epoch 17/20, Train Loss: 0.1708, Validation Loss: 0.6313
Train Precision: 0.89, Train Recall: 0.89, Train F1: 0.89
Validation Precision: 0.77, Validation Recall: 0.77, Validation F1: 0.77


Epoch 18/20: 100% 3631/3631 [20:13<00:00,  2.99batch/s]



Epoch 18/20, Train Loss: 0.1648, Validation Loss: 0.6228
Train Precision: 0.89, Train Recall: 0.89, Train F1: 0.89
Validation Precision: 0.78, Validation Recall: 0.78, Validation F1: 0.78


Epoch 19/20:  27% 998/3631 [05:28<12:27,  3.52batch/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Epoch 20/20, Train Loss: 0.1594, Validation Loss: 0.6470
Train Precision: 0.89, Train Recall: 0.89, Train F1: 0.89
Validation Precision: 0.78, Validation Recall: 0.77, Validation F1: 0.77


In [8]:
model = PAtt_Lite(num_classes=3)  # 클래스 수에 맞게 설정
checkpoint = torch.load('model.pth')

# classification_head를 제외한 모든 레이어의 파라미터를 로드
checkpoint = {k: v for k, v in checkpoint.items() if 'classification_head' not in k}
model.load_state_dict(checkpoint, strict=False)
model.eval()

PAtt_Lite(
  (base_model): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [ ]:
from torchvision import transforms
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# 이미지 로드 및 전처리
image_path = './data/test/0cebc58b878df61c71de8e92bf181af059fe272b432e0ff5bab15d4aa12504f6__20___20201202162128-010-006.jpg'
image = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
image = transform(image).unsqueeze(0)  # 배치 차원 추가

# 예측 수행
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

print(f'Predicted class: {predicted.item()}')

# 클래스 인덱스에 해당하는 실제 이름 매핑 (이 예제에서는 임의로 설정)
class_names = {0: 'negative', 1: 'neutral', 2: 'positive'}

# 예측 수행 및 확률 계산
with torch.no_grad():
    outputs = model(image)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)[0] * 100
    _, predicted = torch.max(outputs, 1)

# 예측된 클래스 이름 및 확률
predicted_class_name = class_names[predicted.item()]
predicted_probability = probabilities[predicted.item()]

# 이미지와 예측 결과 시각화
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# 이미지 시각화
ax[0].imshow(image.squeeze().permute(1, 2, 0))  # 이미지의 차원을 (C, H, W)에서 (H, W, C)로 변경
ax[0].set_title("Prediction")
ax[0].text(10, 20, f'{predicted_class_name}: {predicted_probability:.2f}%', color='red', fontsize=15, bbox=dict(facecolor='white', alpha=0.5))

# 확률 시각화
ax[1].barh(np.arange(len(class_names)), probabilities.numpy(), color='skyblue')
ax[1].set_xlim(0, 100)
ax[1].set_yticks(np.arange(len(class_names)))
ax[1].set_yticklabels(class_names.values())
ax[1].set_title("Class Probabilities")
ax[1].invert_yaxis()  # 확률이 높은 클래스를 위로

plt.tight_layout()
plt.show()


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd

# 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 테스트 디렉토리 및 라벨 파일
test_dir = './data/test/'
label_file = './data/labels_test.csv'

# 라벨 파일 로드
labels_df = pd.read_csv(label_file)
# 라벨 파일에서 filename과 label을 딕셔너리로 변환
test_labels = pd.Series(labels_df.label.values,index=labels_df.filename).to_dict()

# 클래스 이름 매핑
class_names = {'negative': 0, 'neutral': 1, 'positive': 2}

# 정확도 계산을 위한 변수 초기화
correct = 0
total = 0

# 모든 jpg 파일을 읽어서 예측 수행
for filename in os.listdir(test_dir):
    if filename.endswith(".jpg"):
        # 실제 라벨 찾기
        actual_label = class_names[test_labels[filename]]
        image_path = os.path.join(test_dir, filename)
        image = Image.open(image_path)
        image = transform(image).unsqueeze(0)  # 배치 차원 추가

        # 예측 수행
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            total += 1
            correct += (predicted.item() == actual_label)

# 정확도 출력
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


## 